# Modelo Preditivo de Fraude - Marcador de POC (DRAFT)
Random Forest sem Cross validation

## Objetivo
Os POCs reportados são focados em agências com maior ocorrência de fraude, não constituindo, portanto, um registro completo dos POCs ocorridos. Esta análise tem por objetivo  a melhorar a performance do modelo preditivo, a partir da marcação dos POCs de forma mais completa, por meio da análise dos eventos ocorridos nas agências/ATMs com POCs reportados.

## Libraries

In [347]:
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
from pyspark.sql.types import Row
from pyspark.sql.types import *
from pyspark.sql.types import DoubleType
from pyspark.sql.types import DateType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import *
#import matplotlib as mpl
#import matplotlib.pyplot as plt
import atexit
from numpy import array
import numpy as np
import pandas as pd
from pyspark.mllib.linalg import Vectors
from pylab import plot,show,hist,figure,title
import unicodedata
from datetime import datetime, time
from pyspark.sql.functions import col,udf, unix_timestamp
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.sql import HiveContext 
hive_context = HiveContext(sc)
# para plots
%matplotlib inline

## Carga de Dados Brutos Hadoop Produção

In [348]:
# PARA HADOOP PRODUCAO USAR ABAIXO
funcStrToDate_mdyy =  udf (lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')\
                           .replace(minute=0, hour=0, second=0, microsecond=0), DateType())
funcStrToDate_dmyyyy =  udf (lambda x: datetime.strptime(x, '%m/%d/%y'), DateType())

In [349]:
# PARA HADOOP PRODUCAO USAR ABAIXO
rdd_eventlog = hive_context.table('default.ext_eventos_fullog1908')
poc_in = '/user/x181472/export/BASE_POC_ATM_090816.csv'
tranlog_in = hive_context.table('default.ext_juntatudo12')
tabelao_out = '/temporario/tabelao-temp-V5.2.7.csv'
previsao = '/temporario/previsao-V5.2.7.csv'
#tabelao_in = '/user/x181472/export/tabelao-temp-V5.2.7.csv'
tabelao_in = '/user/x184166/tabelao-temp-V5.2.7.csv' # Hermann

In [350]:
# PARA HADOOP PRODUCAO USAR ABAIXO
df_eventlog = rdd_eventlog \
    .map(lambda p: (p[1],p[2],p[3], p[4])) \
    .toDF(['nr_sequ_even',
           'dh_even', 
           'cd_eqpt', 
           'nr_idef_secu' 
                   ])
df_eventlog.printSchema() 

print ("\n\nNumero de linhas carregadas: " + str("%d" % (rdd_eventlog.count())))


root
 |-- nr_sequ_even: long (nullable = true)
 |-- dh_even: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_idef_secu: string (nullable = true)



Numero de linhas carregadas: 18341921


In [212]:
# PARA HADOOP PRODUCAO USAR ABAIXO
# sumarizar a base por agencia+atm+evento+dia
df_eventlog = df_eventlog\
    .groupBy(['nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'dh_even']).count()\
    .orderBy(col('nr_idef_secu'), col('cd_eqpt'), col('nr_sequ_even'), col('dh_even').desc())\
    .withColumnRenamed('count', 'nr_eventos')


## Carga de dados Brutos MAC

In [213]:
# PARA MAC USAR ABAIXO
#eventlog_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tbfullog.csv'
#poc_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/BASE_POC_ATM_090816.csv'
#tranlog_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/dados/juntatudo11pordia-tranfin.csv'
#tabelao_out = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tabelao_stdr-V5.2.2.csv'
#tabelao_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tabelao_stdr-V5.2.2.csv'
#previsao = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/previsao-V5.2.2.csv'

In [214]:
# PARA MAC
#funcStrToDate_mdyy =  udf (lambda x: datetime.strptime(x, '%d/%m/%Y'), DateType())
#funcStrToDate_dmyyyy =  udf (lambda x: datetime.strptime(x, '%m/%d/%y'), DateType())

In [215]:
# PARA MAC USAR ABAIXO
#rdd_eventlog = sc.textFile(eventlog_in)
#print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_eventlog.count())))

#header = rdd_eventlog.first()

#df_eventlog = rdd_eventlog \
#    .filter(lambda line: line != header) \
#    .map(lambda k: k.split(",")) \
#    .map(lambda p: (p[1],p[2],p[3],p[4], p[5])) \
#    .toDF(['nr_sequ_even',
#            'dh_even', 
#            'cd_eqpt', 
#            'nr_idef_secu', 
#            'nr_eventos'
#                   ])
#df_eventlog.printSchema()    

In [216]:
df_eventlog = df_eventlog.withColumn('dh_even', funcStrToDate_mdyy(col('dh_even'))) 

In [217]:
df_eventlog = df_eventlog \
    .withColumn('nr_sequ_even', df_eventlog["nr_sequ_even"].cast(StringType()))\
    .withColumn('cd_eqpt', df_eventlog["cd_eqpt"].cast(StringType()))\
    .withColumn('nr_idef_secu', df_eventlog["nr_idef_secu"].cast(StringType())) \
    .withColumn('dh_even', df_eventlog["dh_even"].cast(TimestampType()))\
    .withColumn('nr_eventos', df_eventlog["nr_eventos"].cast(IntegerType()))

df_eventlog.printSchema()   

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)



In [224]:
# obtem a data mais recente do log de eventos de (2015, 7, 8) a (2016, 12, 1)
# DESCOBERTAS DATAS DO FUTURO NO LOG??? 01-12-2016)
#df_eventlog.orderBy(col('dh_even').desc()).limit(10).toPandas()

nr_idef_secu cd_eqpt nr_sequ_even    dh_even  nr_eventos
0          568     164          718 2016-12-01           1
1          568     164          934 2016-12-01           1
2          568     164         5694 2016-12-01           1
3          568     164          718 2016-12-01           1
4          568     164         5743 2016-12-01           1
5          568     164         5697 2016-12-01           1
6         1640   10001         1846 2016-08-22           1
7         3063   10001         1846 2016-08-22           1
8         2093   10001         1846 2016-08-22           1
9         2286   10001         1846 2016-08-22           1

In [218]:
#df_eventlog.limit(10).orderBy(col("nr_eventos").desc()).toPandas()

In [246]:
# DEPOIS: acertar abaixo para pegar direto ultima data do arquivo original como data_ref
# Filtrando últimos 90 dias
dias_a_subtrair = 90 
import datetime, time
from dateutil.relativedelta import relativedelta

# obtendo faixa do período de histórico a partir da data de referencia
data_ref = datetime.datetime(2016, 8, 8) 
noventa_dias_antes = data_ref + relativedelta(days=-dias_a_subtrair)
print (noventa_dias_antes)
type (noventa_dias_antes)

2016-05-10 00:00:00


datetime.datetime

In [247]:
data_atual = ("2016-08-08")
data_ref = ("2016-05-10")
dt_sql_atual = (time.mktime(datetime.datetime.strptime(data_atual, "%Y-%m-%d").timetuple()))
dt_sql = (time.mktime(datetime.datetime.strptime(data_ref, "%Y-%m-%d").timetuple()))
dt_sql

1462849200.0

In [248]:
# FILTRANDO SOMENTE ULTIMOS 90 DIAS
#df_eventlog = df_eventlog.where("CAST(dh_even AS INT) >= {0}".format(dt_sql)).orderBy("dh_even")
df_eventlog = df_eventlog.where("CAST(dh_even AS INT) BETWEEN {0} AND {1}"\
                                .format(dt_sql,dt_sql_atual)).orderBy("dh_even")
df_eventlog.take(2)

[Row(nr_idef_secu=u'105', cd_eqpt=u'10001', nr_sequ_even=u'569', dh_even=datetime.datetime(2016, 5, 24, 0, 0), nr_eventos=1),
 Row(nr_idef_secu=u'105', cd_eqpt=u'10001', nr_sequ_even=u'569', dh_even=datetime.datetime(2016, 5, 24, 0, 0), nr_eventos=1)]

## Obtendo Eventos Marcadores de POC (selecionados como features)
Usando como baseline o arquivo de POCs reportados, serão identificados os eventos fora do padrão das agências/ATMs para serem usados como marcadores de POC no modelo preditivo.
São considerados fora do padrão, eventos que ocorrem em dias de ocorrência de POC, em agências reportadas como tendo ocorrência de POC, e que têm a quantidade de ocorrências no dia acima do percentil 75% da distribuição diária do eventos em todas as ATMs. Além disso, devem ser eventos que não aparecem com valores maiores que o percentil 75% nos dias em que não existem ocorrências de POCs.

In [249]:
# DUVIDA: tem sentido pegar o percentil e mediana de eventos como um todo ao inves de por Agencia/ATM como faço?
# Por enquanto manter por agencia/atm

# calculando percentis de baseline para todas as observações do log com base em agencia/atm/evento
# OBS: o uso do percentil desta forma permite usar os eventos isoladamente como marcadores 
# pois normaliza-os, tornando este marcador independente das ATM onde ocorrem
df_eventlog_percen = df_eventlog.withColumn('id', concat(col("nr_idef_secu"), lit("-"), 
                                             col("cd_eqpt"), lit("-"),col("nr_sequ_even")))\
                                .select(['id', 'nr_eventos'])

In [233]:
#df_eventlog_percen.take(5)

In [250]:
mediana = 50
percentil_max = 75
rdd_eventlog_percen = df_eventlog_percen.rdd \
    .map(lambda x: (x[0], (x[1],), )) \
    .reduceByKey(lambda acc, val: acc + val) \
    .map(lambda x: (x[0], np.percentile(x[1], mediana), np.percentile(x[1], percentil_max)))

In [251]:
rdd_eventlog_percen = rdd_eventlog_percen \
    .map(lambda k: (k[0].split("-"), k[1], k[2])) 

In [252]:
df_eventlog_percen = rdd_eventlog_percen \
    .map(lambda p: (p[0][0],p[0][1],p[0][2],float(p[1]), float(p[2])  )) \
    .toDF(['nr_idef_secu',
           'cd_eqpt',
           'nr_sequ_even',
           'mediana',
           'percentil_max'
                   ])
#df_eventlog_percen.limit(10).toPandas()

In [253]:
df_eventlog_percen.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- mediana: double (nullable = true)
 |-- percentil_max: double (nullable = true)



In [254]:
#df_eventlog_percen.count()

In [255]:
#df_eventlog_percen.orderBy(col('percentil_max').desc()).limit(10).toPandas()

In [256]:
#df_eventlog_percen.orderBy(col('percentil_max').desc()).filter(col('percentil_max') > col('mediana')).limit(10).toPandas()

In [257]:
# colocar no log total as colunas de mediana e percentil máximo por agencia+atm+evento 
df_eventlog2 = df_eventlog \
    .join(df_eventlog_percen, (df_eventlog['nr_idef_secu'] == df_eventlog_percen['nr_idef_secu']) &
                              (df_eventlog['cd_eqpt'] == df_eventlog_percen['cd_eqpt']) &
                              (df_eventlog['nr_sequ_even'] == df_eventlog_percen['nr_sequ_even']), 'left') \
    .drop(df_eventlog_percen["nr_idef_secu"])\
    .drop(df_eventlog_percen["cd_eqpt"])\
    .drop(df_eventlog_percen["nr_sequ_even"])

In [258]:
#df_eventlog2.count()

In [259]:
# tirando zeros a esquerda e truncando codigos de agencia e atm
df_eventlog2 = df_eventlog2\
    .withColumn('cd_eqpt', trim(df_eventlog2.cd_eqpt))\
    .withColumn('nr_idef_secu', trim(df_eventlog2.nr_idef_secu))\
    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0+(?!$)', '')) # removendo zero `esquerda
#    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0', '')) # removendo zero `esquerda
# testar s.replaceFirst("^0+(?!$)", "")

In [260]:
#df_eventlog2.orderBy(col('mediana').desc()).limit(10).toPandas()

### Dados de registro de POC
Obtém os POCs de Retenção de Cartão, somente para os últimos 90 dias.
Trata as seguintes colunas:
+ data de cadastro do POC
+ código da ATM onde ocorreu o POC
+ código da agência onde ocorreu o POC

Obtém os POCs de Golpe reportados.
Identifica quais eventos que aconteceram nas agências e na data que saíram da faixa de percentil. estes eventos serão os **Marcadores de POC**.

In [261]:
rdd_poc = sc.textFile(poc_in)
print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_poc.count())))

header = rdd_poc.first()
df_poc = rdd_poc \
    .filter(lambda line: line != header) \
    .map(lambda k: k.split(",")) \
    .map(lambda p: (p[0],p[2],p[4], p[8])) \
    .toDF(['data_cad', 
            'atm',
            'agencia',
            'golpe'
        ])

df_poc = df_poc.withColumn('data_cad', funcStrToDate_dmyyyy(col('data_cad'))) 

df_poc = df_poc \
    .withColumn('atm', df_poc["atm"].cast(StringType()))\
    .withColumn('agencia', df_poc["agencia"].cast(StringType())) \
    .withColumn('data_cad', df_poc["data_cad"].cast(TimestampType())) \
    .withColumn('golpe', df_poc["golpe"].cast(StringType())) \

# FILTRANDO POCS SOMENTE DOS ULTIMOS 90 DIAS E SOMENTE PARA GOLPE DE RETENCAO DE CARTAO
#df_poc = df_poc.where("golpe = 'GOLPE RETENTOR DE CARTAO' and \
#            CAST(data_cad AS INT) >= {0}".format(dt_sql)).orderBy('agencia', 'data_cad')
df_poc = df_poc.where("CAST(data_cad AS INT) >= {0}".format(dt_sql)).orderBy('agencia', 'data_cad')



Número de linhas carregadas: 1019


In [262]:
# tirando zeros a esquerda e truncando codigos de agencia e atm
df_poc = df_poc\
    .withColumn('atm', trim(df_poc.atm))\
    .withColumn('agencia', trim(df_poc.agencia))\
    .withColumn('agencia', regexp_replace('agencia', '^0+(?!$)', '')) # removendo zero `esquerda
#df_poc.limit(10).toPandas()

In [263]:
df_poc.orderBy(col('data_cad').desc()).toPandas()

data_cad   atm     agencia                       golpe
0   2016-08-08               112    GOLPE RETENTOR DE CARTAO
1   2016-08-08               159  GOLPE RETENTOR DE ENVELOPE
2   2016-08-08               140  GOLPE RETENTOR DE ENVELOPE
3   2016-08-08                59  GOLPE RETENTOR DE ENVELOPE
4   2016-08-08               204  GOLPE RETENTOR DE ENVELOPE
5   2016-08-08               142  GOLPE RETENTOR DE ENVELOPE
6   2016-08-08  "151         153                        3316
7   2016-08-08  "155         157                  167 e 168"
8   2016-08-08  "151  158 e 162"          SEGURANÇA 24 HORAS
9   2016-08-08              3547  GOLPE RETENTOR DE ENVELOPE
10  2016-08-08               915  GOLPE RETENTOR DE ENVELOPE
11  2016-08-08               985  GOLPE RETENTOR DE ENVELOPE
12  2016-08-08              2065  GOLPE RETENTOR DE ENVELOPE
13  2016-08-08              3940  GOLPE RETENTOR DE ENVELOPE
14  2016-08-08              4268    GOLPE RETENTOR DE CARTAO
15  2016-08-08              4550    GOLPE RETENTOR DE CARTAO
16  2016-08-08              4550    GOLPE RETENTOR DE CARTAO
17  2016-08-08              4281  GOLPE RETENTOR DE ENVELOPE
18  2016-08-06               935              TELEFONE FALSO
19  2016-08-06              1561    GOLPE RETENTOR DE CARTAO
20  2016-08-06              4263  GOLPE RETENTOR DE ENVELOPE
21  2016-08-06              1074              TELEFONE FALSO
22  2016-08-06              3898  GOLPE RETENTOR DE ENVELOPE
23  2016-08-06               995  GOLPE RETENTOR DE ENVELOPE
24  2016-08-06              4776  GOLPE RETENTOR DE ENVELOPE
25  2016-08-06              4268    GOLPE RETENTOR DE CARTAO
26  2016-08-04     0        3989  GOLPE RETENTOR DE ENVELOPE
27  2016-08-04     0         787  GOLPE RETENTOR DE ENVELOPE
28  2016-08-04     0         787    GOLPE RETENTOR DE CARTAO
29  2016-08-04     0        3146    GOLPE RETENTOR DE CARTAO
..         ...   ...         ...                         ...
354 2016-05-16              1207  GOLPE RETENTOR DE ENVELOPE
355 2016-05-16              1227    GOLPE RETENTOR DE CARTAO
356 2016-05-16              3717  GOLPE RETENTOR DE ENVELOPE
357 2016-05-16               118  GOLPE RETENTOR DE ENVELOPE
358 2016-05-16               578  GOLPE RETENTOR DE ENVELOPE
359 2016-05-16              2082    GOLPE RETENTOR DE CARTAO
360 2016-05-16              2082    GOLPE RETENTOR DE CARTAO
361 2016-05-16              4433                            
362 2016-05-16              4395         LEITORAS OBSTRUIDAS
363 2016-05-14                86                         POC
364 2016-05-14               277  GOLPE RETENTOR DE ENVELOPE
365 2016-05-14               259  GOLPE RETENTOR DE ENVELOPE
366 2016-05-14               340  GOLPE RETENTOR DE ENVELOPE
367 2016-05-14              1093  GOLPE RETENTOR DE ENVELOPE
368 2016-05-14              4433         LEITORAS OBSTRUIDAS
369 2016-05-14               264                         POC
370 2016-05-14               570         TECLADOS OBSTRUÍDOS
371 2016-05-14              3987         TECLADOS OBSTRUÍDOS
372 2016-05-13              2286                         POC
373 2016-05-13               909  GOLPE RETENTOR DE ENVELOPE
374 2016-05-13              3066                         POC
375 2016-05-12              1641                         POC
376 2016-05-12              2286                         POC
377 2016-05-11              4425         LEITORAS OBSTRUIDAS
378 2016-05-11               114  GOLPE RETENTOR DE ENVELOPE
379 2016-05-11              4424              TELEFONE FALSO
380 2016-05-11              4279  GOLPE RETENTOR DE ENVELOPE
381 2016-05-11              3733         LEITORAS OBSTRUIDAS
382 2016-05-10               245                         POC
383 2016-05-10              3385                         POC

[384 rows x 4 columns]

In [264]:
df_eventlog2.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)
 |-- mediana: double (nullable = true)
 |-- percentil_max: double (nullable = true)



In [60]:
# DESNECESSARIA A PESQUISA ABAIXO AGORA

#df_eventlog2.registerTempTable("tb_logeven")
#df_poc.registerTempTable("tb_pocagen")

#df_evenpoc = sqlContext.sql("select * from tb_logeven where exists \
#                        (select agencia from tb_pocagen where tb_logeven.nr_idef_secu = tb_pocagen.agencia)")

# TESTAR AQUI SE SELECIONOU SOMENTE AGENCIAS DO POC
# tem que trazer igual ou menor ao numero de agencias do df_poc
#df_evenpoc.groupBy('nr_idef_secu').count().orderBy('nr_idef_secu').toPandas()

## Selecionando Eventos por ATM/Agência em dias POC

In [265]:
# SELECIONANDO DO LOG TOTAL SOMENTE AGENCIAS E DIAS DE OCORRENCIA DE POC ACIMA DO PERCENTIL
df_poceven = df_eventlog2 \
    .join(df_poc, (df_poc['data_cad'] == df_eventlog2['dh_even']) &
                (df_poc['agencia'] == df_eventlog2['nr_idef_secu']), 'left')
#df_poceven.limit(10).toPandas()

In [266]:
# LOG SOMENTE COM AGENCIAS POC NAS DATAS POC E MAIOR QUE PERCENTIL
df_poceven2 = df_poceven\
    .filter((col("nr_eventos") > col('percentil_max')) &
            ((col("agencia") == col('nr_idef_secu'))))

#df_poceven2.limit(10).toPandas()

## Selecionando Eventos por ATM/Agência Fora de dias POC

In [267]:
# LOG SOMENTE COM TODAS AS AGENCIAS NAS DATAS FORA DE POC E MENOR QUE PERCENTIL
df_pocevenNAO = df_poceven\
    .filter((col("nr_eventos") <= col('percentil_max')) &
            (col('agencia').isNull()))
#    .filter(col('agencia').isNull())
#df_pocevenNAO.limit(10).toPandas()

In [268]:
# JOIN DAS DUAS ACIMA POR EVENTO PARA OBTER LISTA DE POKEVENTOS
# DEVE SER LEFT PARA PEGAR EVENTOS POC QUE NAO OCORREM FORA DO POC
df_pokeventos = df_poceven2\
    .join(df_pocevenNAO,
          (df_poceven2['nr_sequ_even'] == df_pocevenNAO['nr_sequ_even']), 'left')\
    .drop(df_pocevenNAO["nr_sequ_even"])\
    .groupBy('nr_sequ_even').count()
    
#df_pokeventos.limit(10).toPandas()
#    .drop(df_eventlog_percen["nr_idef_secu"])\
#    .drop(df_eventlog_percen["cd_eqpt"])\
#    .drop(df_eventlog_percen["nr_sequ_even"])

In [65]:
# CHECK
#df_pokeventos.limit(10).toPandas()

nr_sequ_even  count
0         5610    942
1         5603  37795
2         5620    907
3         5640    951
4         5605    944
5         5616    944

In [269]:
# LISTA DE EVENTOS QUE APARECEM COM VALOR MAIOR QUE PERCENTIL EN DIA DE POC E
# VALOR ABAIXO DO PERCENTIL EM DIAS FORA DE POC
l_evenpoc = df_pokeventos.toPandas().nr_sequ_even.tolist()
l_evenpoc

[u'5603',
 u'5605',
 u'5610',
 u'5612',
 u'5614',
 u'5616',
 u'5620',
 u'5622',
 u'5640']

## Preparando dados processados
Marcando como POC as entradas do log para os eventos marcadores 

In [270]:
# CHECK
#df_eventlog2.limit(10).toPandas()

In [291]:
# APROXIMACAO ABAIXO PARA EVITAR PROBLEMAS NO withColumns
# criar coluna de indice de proximidade da mediana (iproxmed)
# calculado como número de ocorrências real diaria/mediana diaria de eventos por agencia e atm
# If you use deimcal, you need to import

#from decimal import getcontext, Decimal
# Set the precision.
#getcontext().prec = 3

#format(math.pi, '.2f')
# mudar o arredondamento abaixo para ver se afeta o modelo
df_feat = df_eventlog2\
    .withColumn('iproxmed', (col("nr_eventos")/col("mediana"))*1000)
#    .withColumn('iproxmed', round(col("nr_eventos")/col("mediana"),2))
    #df_feat.limit(10).toPandas()

In [292]:
df_feat.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)
 |-- mediana: double (nullable = true)
 |-- percentil_max: double (nullable = true)
 |-- iproxmed: double (nullable = true)



In [272]:
# CHECK
#df_feat.orderBy(col('iproxmed').desc().limit(10).toPandas()

In [70]:
# checagem de arredondamento
#df_tidy3 = df_feat..select('nr_idef_secu', 'iproxmed').filter(col('nr_idef_secu') == '105')
#df_tidy3.collect()
#df_tidy3.toPandas().to_csv('feat-prec3.csv')

In [71]:
#import math
#from functools import reduce
#def arredonda(*col):
#    return reduce(lambda a: format(a, '.2f') ,col)

#arred = udf(arredonda)

#df_tidy2 = df_feat.withColumn('max_evento', arred(df_feat.columns[-1]))

In [72]:
#import math
#format(math.pi, '.2f')

#df_feat.columns[-1]

In [293]:
# transpondo tipo de evento para gerar features
df_feat = df_feat\
    .drop(df_feat["mediana"])

In [294]:
#df_feat.groupBy(['nr_idef_secu']).count().toPandas()

In [295]:
# FILTRANDO SOMENTE FEATURES DE POKEVENTOS
df_feat = df_feat.filter(col("nr_sequ_even").isin(l_evenpoc))

In [296]:
df_feat.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)
 |-- percentil_max: double (nullable = true)
 |-- iproxmed: double (nullable = true)



In [297]:
# OBS: selecionar somente os eventos usados no mllib como features para usar menos memoria:
pd_pivot = df_feat\
    .select('dh_even', 'nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'iproxmed').toPandas()

In [102]:
pd_pivot.dtypes

dh_even         datetime64[ns]
nr_idef_secu            object
cd_eqpt                 object
nr_sequ_even            object
iproxmed               float64
dtype: object

In [78]:
#pd_pivot

dh_even nr_idef_secu cd_eqpt nr_sequ_even  iproxmed
0     2016-05-23          787   10001         5610       1.0
1     2016-05-29          787   10001         5610       1.0
2     2016-06-02          787   10001         5610       1.0
3     2016-06-07          787   10001         5610       1.0
4     2016-06-12          787   10001         5610       1.0
5     2016-06-17          787   10001         5610       1.0
6     2016-06-23          787   10001         5610       1.0
7     2016-06-28          787   10001         5610       1.0
8     2016-07-02          787   10001         5610       1.0
9     2016-07-08          787   10001         5610       1.0
10    2016-07-15          787   10001         5610       1.0
11    2016-07-20          787   10001         5610       1.0
12    2016-07-25          787   10001         5610       1.0
13    2016-07-30          787   10001         5610       1.0
14    2016-08-03          787   10001         5610       1.0
15    2016-08-08          787   10001         5610       1.0
16    2016-08-13          787   10001         5610       1.0
17    2016-08-19          787   10001         5610       1.0
18    2016-05-24          787   10001         5610       1.0
19    2016-06-01          787   10001         5610       1.0
20    2016-06-05          787   10001         5610       1.0
21    2016-06-10          787   10001         5610       1.0
22    2016-06-15          787   10001         5610       1.0
23    2016-06-21          787   10001         5610       1.0
24    2016-06-25          787   10001         5610       1.0
25    2016-07-01          787   10001         5610       1.0
26    2016-07-06          787   10001         5610       1.0
27    2016-05-25          787   10001         5610       1.0
28    2016-05-30          787   10001         5610       1.0
29    2016-06-04          787   10001         5610       1.0
...          ...          ...     ...          ...       ...
42886 2016-08-08         2093   10001         5605       1.0
42887 2016-08-13         2093   10001         5605       1.0
42888 2016-06-15         2093   10001         5605       1.0
42889 2016-06-25         2093   10001         5605       1.0
42890 2016-07-06         2093   10001         5605       1.0
42891 2016-07-11         2093   10001         5605       1.0
42892 2016-07-19         2093   10001         5605       1.0
42893 2016-07-23         2093   10001         5605       1.0
42894 2016-07-28         2093   10001         5605       1.0
42895 2016-08-09         2093   10001         5605       1.0
42896 2016-05-27         2093   10001         5605       1.0
42897 2016-05-31         2093   10001         5605       1.0
42898 2016-06-20         2093   10001         5605       1.0
42899 2016-06-26         2093   10001         5605       1.0
42900 2016-07-05         2093   10001         5605       1.0
42901 2016-07-27         2093   10001         5605       1.0
42902 2016-08-06         2093   10001         5605       1.0
42903 2016-08-10         2093   10001         5605       1.0
42904 2016-08-16         2093   10001         5605       1.0
42905 2016-08-19         2093   10001         5605       1.0
42906 2016-08-15         2093   10001         5605       1.0
42907 2016-07-10         2093   10001         5605       1.0
42908 2016-07-14         2093   10001         5605       1.0
42909 2016-07-17         2093   10001         5605       1.0
42910 2016-07-21         2093   10001         5605       1.0
42911 2016-07-26         2093   10001         5605       1.0
42912 2016-07-31         2093   10001         5605       1.0
42913 2016-08-02         2093   10001         5605       1.0
42914 2016-08-07         2093   10001         5605       1.0
42915 2016-08-14         2093   10001         5605       1.0

[42916 rows x 5 columns]

In [299]:
# transpondo evento por tipo OK! 
# OBS: a funcao .reset_index abaixo é necessário para evitar a mesclagem das células fixas
pd_feat = pd.pivot_table(pd_pivot,index=["nr_idef_secu","cd_eqpt","dh_even"],
                         columns=["nr_sequ_even"], margins=False, values=["iproxmed"], fill_value=0).reset_index()
# acertando nomes das colunas geradas em linhas diferentes

In [300]:
pd_feat.columns = [c[1] if c[1] else c[0] for c in pd_feat.columns.tolist()]

In [301]:
# OBS. por algum motivo, pivot_table está transformando float em long
# solucao temporaria: multiplicar aproxmed por 1000 antes e depois transformar em double e dividir por 1000
# TENTAR ABAIXO
#aggfunc = lambda x: x
pd_feat.dtypes

nr_idef_secu            object
cd_eqpt                 object
dh_even         datetime64[ns]
5603                   float64
5605                     int64
5610                     int64
5612                     int64
5614                     int64
5616                     int64
5620                     int64
5622                     int64
5640                     int64
dtype: object

In [302]:
pd_feat

nr_idef_secu cd_eqpt    dh_even    5603  5605  5610  5612  5614  5616  \
0           1053   10001 2016-05-24  1000.0  1000  1000  1000  1000  1000   
1           1053   10001 2016-05-25  1000.0  1000  1000  1000  1000  1000   
2           1053   10001 2016-05-26  1000.0  1000  1000  1000  1000  1000   
3           1053   10001 2016-05-27  1000.0  1000  1000  1000  1000  1000   
4           1053   10001 2016-05-28  1000.0  1000  1000  1000  1000  1000   
5           1053   10001 2016-05-29  1000.0  1000  1000  1000  1000  1000   
6           1053   10001 2016-05-30  1000.0  1000  1000  1000  1000  1000   
7           1053   10001 2016-05-31  1000.0  1000  1000  1000  1000  1000   
8           1053   10001 2016-06-01  1000.0  1000  1000  1000  1000  1000   
9           1053   10001 2016-06-02  1000.0  1000  1000  1000  1000  1000   
10          1053   10001 2016-06-03  1000.0  1000  1000  1000  1000  1000   
11          1053   10001 2016-06-04  1000.0  1000  1000  1000  1000  1000   
12          1053   10001 2016-06-05  1000.0  1000  1000  1000  1000  1000   
13          1053   10001 2016-06-06  1000.0  1000  1000  1000  1000  1000   
14          1053   10001 2016-06-07  1000.0  1000  1000  1000  1000  1000   
15          1053   10001 2016-06-08  1000.0  1000  1000  1000  1000  1000   
16          1053   10001 2016-06-09  1000.0  1000  1000  1000  1000  1000   
17          1053   10001 2016-06-10  1000.0  1000  1000  1000  1000  1000   
18          1053   10001 2016-06-11  1000.0  1000  1000  1000  1000  1000   
19          1053   10001 2016-06-12  1000.0  1000  1000  1000  1000  1000   
20          1053   10001 2016-06-13  1000.0  1000  1000  1000  1000  1000   
21          1053   10001 2016-06-14  1000.0  1000  1000  1000  1000  1000   
22          1053   10001 2016-06-15  1000.0  1000  1000  1000  1000  1000   
23          1053   10001 2016-06-16  1000.0  1000  1000  1000  1000  1000   
24          1053   10001 2016-06-17  1000.0  1000  1000  1000  1000  1000   
25          1053   10001 2016-06-18  1000.0  1000  1000  1000  1000  1000   
26          1053   10001 2016-06-19  1000.0  1000  1000  1000  1000  1000   
27          1053   10001 2016-06-20  1000.0  1000  1000  1000  1000  1000   
28          1053   10001 2016-06-21  1000.0  1000  1000  1000  1000  1000   
29          1053   10001 2016-06-22  1000.0  1000  1000  1000  1000  1000   
..           ...     ...        ...     ...   ...   ...   ...   ...   ...   
690          787   10001 2016-07-10  1000.0  1000  1000  1000  1000  1000   
691          787   10001 2016-07-11  1000.0  1000  1000  1000  1000  1000   
692          787   10001 2016-07-12  1000.0  1000  1000  1000  1000  1000   
693          787   10001 2016-07-13  1000.0  1000  1000  1000  1000  1000   
694          787   10001 2016-07-14  1000.0  1000  1000  1000  1000  1000   
695          787   10001 2016-07-15  1000.0  1000  1000  1000  1000  1000   
696          787   10001 2016-07-16  1000.0  1000  1000  1000  1000  1000   
697          787   10001 2016-07-17  1000.0  1000  1000  1000  1000  1000   
698          787   10001 2016-07-18  1000.0  1000  1000  1000  1000  1000   
699          787   10001 2016-07-19  1000.0  1000  1000  1000  1000  1000   
700          787   10001 2016-07-20  1000.0  1000  1000  1000  1000  1000   
701          787   10001 2016-07-21  1000.0  1000  1000  1000  1000  1000   
702          787   10001 2016-07-22  1000.0  1000  1000  1000  1000  1000   
703          787   10001 2016-07-23  1000.0  1000  1000  1000  1000  1000   
704          787   10001 2016-07-24  1000.0  1000  1000  1000  1000  1000   
705          787   10001 2016-07-25  1000.0  1000  1000  1000  1000  1000   
706          787   10001 2016-07-26  1000.0  1000  1000  1000  1000  1000   
707          787   10001 2016-07-27  1000.0  1000  1000  1000  1000  1000   
708          787   10001 2016-07-28  1000.0  1000  1000  1000  1000  1000   
709          787   10001 2016-07-29  1000.0  1000  1000  1000  1000  1000   


In [81]:
#c = pd_feat.columns.tolist
#c[1]

In [303]:
# criar coluna com data como string para posterior transformação para dataframe
pd_feat = pd_feat.assign(data_even = lambda x: x['dh_even'].dt.strftime('%Y-%m-%d') ).drop('dh_even', axis=1)
pd_feat

nr_idef_secu cd_eqpt    5603  5605  5610  5612  5614  5616  5620  5622  \
0           1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
1           1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
2           1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
3           1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
4           1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
5           1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
6           1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
7           1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
8           1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
9           1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
10          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
11          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
12          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
13          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
14          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
15          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
16          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
17          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
18          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
19          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
20          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
21          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
22          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
23          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
24          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
25          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
26          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
27          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
28          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
29          1053   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
..           ...     ...     ...   ...   ...   ...   ...   ...   ...   ...   
690          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
691          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
692          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
693          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
694          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
695          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
696          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
697          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
698          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
699          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
700          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
701          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
702          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
703          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
704          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
705          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
706          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
707          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
708          787   10001  1000.0  1000  1000  1000  1000  1000  1000  1000   
709          787   10001  

In [83]:
# salva como csv pois conversao para Dataframe Spark apresenta erro. provavel falta de memoria
#pd_feat.to_csv(tabelao_out)

In [84]:
pd_feat.columns

Index([u'nr_idef_secu',      u'cd_eqpt',         u'5603',         u'5605',
               u'5610',         u'5616',         u'5620',         u'5640',
          u'data_even'],
      dtype='object')

In [304]:
df_feat_alt = sqlCtx.createDataFrame(pd_feat)
# FALTA converter data em timedate de novo (converter para string no Pandas antes)

In [305]:
df_feat_alt.take(5)

[Row(nr_idef_secu=u'1217', cd_eqpt=u'10001', 5603=1000.0, 5605=1000, 5610=1000, 5612=1000, 5614=1000, 5616=1000, 5620=1000, 5622=1000, 5640=1000, data_even=u'2016-07-30'),
 Row(nr_idef_secu=u'1217', cd_eqpt=u'10001', 5603=1000.0, 5605=1000, 5610=1000, 5612=1000, 5614=1000, 5616=1000, 5620=1000, 5622=1000, 5640=1000, data_even=u'2016-07-31'),
 Row(nr_idef_secu=u'1217', cd_eqpt=u'10001', 5603=1000.0, 5605=1000, 5610=1000, 5612=1000, 5614=1000, 5616=1000, 5620=1000, 5622=1000, 5640=1000, data_even=u'2016-08-01'),
 Row(nr_idef_secu=u'1217', cd_eqpt=u'10001', 5603=1000.0, 5605=1000, 5610=1000, 5612=1000, 5614=1000, 5616=1000, 5620=1000, 5622=1000, 5640=1000, data_even=u'2016-08-02'),
 Row(nr_idef_secu=u'1217', cd_eqpt=u'10001', 5603=1000.0, 5605=1000, 5610=1000, 5612=1000, 5614=1000, 5616=1000, 5620=1000, 5622=1000, 5640=1000, data_even=u'2016-08-03')]

In [94]:
#df_feat_alt\
#   .withColumn('5605', df_feat_alt["5605"].cast(DoubleType()))\
#    .withColumn('5610', df_feat_alt["5610"].cast(DoubleType()))\
#    .withColumn('5616', df_feat_alt["5616"].cast(DoubleType()))\
#    .withColumn('5605', df_feat_alt["5605"].cast(DoubleType()))\
#    .withColumn('5605', df_feat_alt["5605"].cast(DoubleType()))\

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 5603: double (nullable = true)
 |-- 5605: long (nullable = true)
 |-- 5610: long (nullable = true)
 |-- 5616: long (nullable = true)
 |-- 5620: long (nullable = true)
 |-- 5640: long (nullable = true)
 |-- data_even: string (nullable = true)



In [53]:
df_feat_alt.limit(10).toPandas()

In [103]:
# OBS: se já tiver o tabelao_alt gerado, pode rodar a partir daqui!!!!!
#############################################################################
# AQUI TIVE QUE FAZER ESTA GAMBIARRA: salvar como csv e alguma manipulaçao (acertar cabeçalho) para depois ler
# até resolver o problema de memoria gerado por fazer o pivot no Pandas
#rdd_feat_alt = sc.textFile(tabelao_in)

In [104]:
#rdd_feat_alt.first()
#df = sqlContext.load(source="com.databricks.spark.csv", header="true", path = "cars.csv")

In [105]:
#print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_feat_alt.count())))

In [106]:
#header = rdd_feat_alt.first()
#chemaString = header
#fields = [StructField(field_name, StringType(), False) for field_name in schemaString.split(',')]
#schema = StructType(fields)

#rdd_feat_alt = rdd_feat_alt \
#    .filter(lambda line: line != header) \
#    .map(lambda k: k.split(","))

#df_feat_alt = sqlContext.createDataFrame(rdd_feat_alt, schema)

In [107]:
df_feat_alt.limit(10).toPandas()

nr_idef_secu cd_eqpt  5603  5605  5610  5616  5620  5640   data_even
0         4212   10001   1.0     0     0     0     0     0  2016-05-19
1         4212   10001   1.0     0     0     0     0     0  2016-05-20
2         4212   10001   1.0     1     1     1     1     1  2016-05-21
3         4212   10001   1.0     1     1     1     1     1  2016-05-22
4         4212   10001   1.0     1     1     1     1     1  2016-05-23
5         4212   10001   1.0     1     1     1     1     1  2016-05-24
6         4212   10001   1.0     1     1     1     1     1  2016-05-25
7         4212   10001   1.0     1     1     1     1     1  2016-05-26
8         4212   10001   1.0     1     1     1     1     1  2016-05-27
9         4212   10001   1.0     1     1     1     1     1  2016-05-28

In [108]:
#df_feat_alt.printSchema()

In [306]:
df_feat_alt = df_feat_alt.withColumn('data_even', df_feat_alt["data_even"].cast(TimestampType()))
#df_feat_alt.limit(10).toPandas()

In [307]:
df_feat_alt = df_feat_alt\
    .withColumn('nr_idef_secu', trim(df_feat_alt.nr_idef_secu))
#df_feat_alt.printSchema()

In [308]:
#df_feat_alt.printSchema()

In [309]:
# primeiro left_join df_feat_alt + df_poc
    # depois para agencia == cd_secu and cada coluna evento > 1 = POC
df_tidy = df_feat_alt \
    .join(df_poc, (df_feat_alt['nr_idef_secu'] == df_poc['agencia']) &
            (df_feat_alt['data_even'] == df_poc['data_cad']), 'left')
#df_tidy.limit(10).toPandas()

In [113]:
df_tidy.orderBy(col('agencia').desc())\
    .filter(col('agencia') == col('nr_idef_secu'))\
    .select('agencia').distinct().toPandas()
# 53 agências distintas com POC

agencia
0     787
1    3350
2    3063
3    1217
4    1053

In [114]:
df_tidy.limit(10).toPandas()

nr_idef_secu cd_eqpt  5603  5605  5610  5616  5620  5640  data_even  \
0         1053   10001   1.0     1     1     1     1     1 2016-06-07   
1         1217   10001   1.0     1     1     1     1     1 2016-05-24   
2         4212   10001   1.0     0     0     0     0     0 2016-05-18   
3         1053   10001   1.0     1     1     1     1     1 2016-07-04   
4         1053   10001   1.0     1     1     1     1     1 2016-07-08   
5         1217   10001   1.0     0     0     0     0     0 2016-05-12   
6         1217   10001   1.0     1     1     1     1     1 2016-06-14   
7         1217   10001   1.0     1     1     1     1     1 2016-06-16   
8         3063   10001   1.0     1     1     1     1     1 2016-05-24   
9         3454   10001   1.0     1     1     1     1     1 2016-08-16   

  data_cad   atm agencia golpe  
0     None  None    None  None  
1     None  None    None  None  
2     None  None    None  None  
3     None  None    None  None  
4     None  None    None  None  
5     None  None    None  None  
6     None  None    None  None  
7     None  None    None  None  
8     None  None    None  None  
9     None  None    None  None

In [310]:
# Transformar em double as features (features selecionadas)
for even in l_evenpoc:
    df_tidy_dbl = df_tidy.withColumn(even, df_tidy[even].cast(DoubleType()))

In [316]:
df_tidy_dbl.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 5603: double (nullable = true)
 |-- 5605: double (nullable = true)
 |-- 5610: double (nullable = true)
 |-- 5612: double (nullable = true)
 |-- 5614: double (nullable = true)
 |-- 5616: double (nullable = true)
 |-- 5620: double (nullable = true)
 |-- 5622: double (nullable = true)
 |-- 5640: double (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- data_cad: timestamp (nullable = true)
 |-- atm: string (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)



In [315]:
df_tidy_dbl = df_tidy\
    .withColumn('5603', df_tidy['5603'].cast(DoubleType()))\
    .withColumn('5605', df_tidy['5605'].cast(DoubleType()))\
    .withColumn('5610', df_tidy['5610'].cast(DoubleType()))\
    .withColumn('5612', df_tidy['5612'].cast(DoubleType()))\
     .withColumn('5614', df_tidy['5614'].cast(DoubleType()))\
     .withColumn('5616', df_tidy['5616'].cast(DoubleType()))\
    .withColumn('5620', df_tidy['5620'].cast(DoubleType()))\
    .withColumn('5622', df_tidy['5622'].cast(DoubleType()))\
     .withColumn('5640', df_tidy['5640'].cast(DoubleType()))

In [317]:
# MARCA POC PARA AGENCIAS QUE OCORREM NA TABELA POC, NA DATA POC E QUE AO MENOS UM EVENTO(FEATURE)
# TENHA VALOR MAIOR QUE 1 (INDICANDO OCORRENCIA ACIMA DA MEDIANA)

##############################################################################
# POSSIVEL MELHORIA EM FASE 2: ao inves de label, criar target para regressao, com valor igual
# ao maximo do valor de quantos eventos ocorrem acima de 1 nas features
##############################################################################
# antes separar somente as colunas de eventos
from functools import reduce
def superMax(*cols):
    return reduce(lambda a, b: __builtin__.max(a,b) ,cols)

add = udf(superMax)

df_tidy_dbl = df_tidy_dbl.withColumn('max_evento', add(*[df_tidy_dbl[x] for x in df_tidy_dbl.columns[3:-8]]))
# tentar cast par double acima    

In [318]:
df_tidy_dbl.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 5603: double (nullable = true)
 |-- 5605: double (nullable = true)
 |-- 5610: double (nullable = true)
 |-- 5612: double (nullable = true)
 |-- 5614: double (nullable = true)
 |-- 5616: double (nullable = true)
 |-- 5620: double (nullable = true)
 |-- 5622: double (nullable = true)
 |-- 5640: double (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- data_cad: timestamp (nullable = true)
 |-- atm: string (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)
 |-- max_evento: string (nullable = true)



In [319]:
# POSSIVEL PROBLEMA NA CONVERSAO DE STRING PARA DOUBLE NO USO DE withColumn
df_tidy_dbl = df_tidy_dbl.withColumn('max_evento', df_tidy_dbl["max_evento"].cast(DoubleType()))

In [320]:
df_tidy_dbl.take(2)

[Row(nr_idef_secu=u'1053', cd_eqpt=u'10001', 5603=1000.0, 5605=1000.0, 5610=1000.0, 5612=1000.0, 5614=1000.0, 5616=1000.0, 5620=1000.0, 5622=1000.0, 5640=1000.0, data_even=datetime.datetime(2016, 8, 4, 0, 0), data_cad=None, atm=None, agencia=None, golpe=None, max_evento=1000.0),
 Row(nr_idef_secu=u'1217', cd_eqpt=u'10001', 5603=1000.0, 5605=1000.0, 5610=1000.0, 5612=1000.0, 5614=1000.0, 5616=1000.0, 5620=1000.0, 5622=1000.0, 5640=1000.0, data_even=datetime.datetime(2016, 6, 20, 0, 0), data_cad=None, atm=None, agencia=None, golpe=None, max_evento=1000.0)]

In [321]:
# TENTATIVA 1 (FUCIONOU COM df_tidy!!????) NAO funciona com comparacoa com double no MAC??? versao spark
df_tidy_dbl = df_tidy_dbl.withColumn("label",
    when((df_tidy_dbl["max_evento"] > 1000.0), 1.0)\
        .otherwise(0.0))

In [322]:
df_tidy_dbl.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 5603: double (nullable = true)
 |-- 5605: double (nullable = true)
 |-- 5610: double (nullable = true)
 |-- 5612: double (nullable = true)
 |-- 5614: double (nullable = true)
 |-- 5616: double (nullable = true)
 |-- 5620: double (nullable = true)
 |-- 5622: double (nullable = true)
 |-- 5640: double (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- data_cad: timestamp (nullable = true)
 |-- atm: string (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)
 |-- max_evento: double (nullable = true)
 |-- label: double (nullable = false)



In [110]:
#df_tidy3 = df_tidy2.select('nr_idef_secu', '934', 'maxevento')

In [111]:
#df_tidy3.toPandas().to_csv('feat.csv')

In [112]:
#df = pd.read_csv('feat.csv')

In [113]:
# TENTATIVA 1 (FUNCIONOU COM df_tidy!!????) NAO funciona com comparacoa com double???
#df_tidy3.withColumn("label",
#    when(col('maxevento') > 1.0, 1.0)\
#        .otherwise(0.0)).take(2)

In [114]:
# testar truncar o valor do coluna max_evento
#df_tidy3.withColumn("label",
#    when((df_tidy3['max_evento'] == 1.0), 1.0)\
#        .otherwise(0.0)).take(2)

In [115]:
# TENTATIVA 3
#function = udf(lambda even: 1 if even > 1.0 else 0, IntegerType())

#df_tidy2.select('nr_idef_secu', 'cd_eqpt', 'data_even', 'max_evento', function(col('max_evento')).alias('label')).show()

In [116]:
#df_tidy.columns[3:-8]

In [117]:
# TENTATIVA 2
#from pyspark.sql.functions import udf
#from pyspark.sql.types import IntegerType
#from math import *

#def marcaPOC (max_even):
#    if max_even > 1.0:
#        return 1.0
#    else:
#        return 0.0

#POCmarcado = udf(marcaPOC, FloatType())

#df_tidy2.withColumn("label",POCmarcado(df_tidy2['max_evento'])).take(2)

In [118]:
# TENTATIVA 4
#rdd1 = df_tidy2.rdd.map(lambda row:
#   Row(row.__fields__ + ["label"])(row + (1.0, )))
#rdd1.take(2)

In [119]:
#header = rdd1.first()
#schemaString = header
#fields = [StructField(field_name, StringType(), False) for field_name in schemaString.split(',')]
#schema = StructType(fields)

#rdd1 = rdd1 \
#    .filter(lambda line: line != header) \
#    .map(lambda k: k.split(","))

#df_feat_alt = sqlContext.createDataFrame(rdd1, schema)

In [120]:
#df.rdd.map(lambda row:
#    Row(row.__fields__ + ["day"])(row + (row.date_time.day, ))
#)

In [121]:
#df_tidy3.groupBy("max_evento").count().orderBy("max_evento").show()

In [123]:
#POR USAR VERSAO > x.x.x. do SPARK
#df_tidy3\
#    .withColumn("label",
#    when(df_tidy3['max_evento'] > 1.0, 1.0)\
#        .otherwise(0.0)).take(3)
#df_tidy3.filter(col('max_evento') > 1.0)

In [124]:
#df_tidy3.limit(10).toPandas()
#df_tidy4 = df_tidy3\

In [323]:
df_tidy_dbl.groupBy("label").count().orderBy("label").show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|  710|
|  1.0|   11|
+-----+-----+



In [165]:
# tabelao com labels
#df_tidy.toPandas().to_csv(tabelao_label)

In [166]:
#df_tidy.printSchema()

In [167]:
# ATANCAO: ver se falta fazer antes
#df_tidy = df_feat_alt\
#   .withColumn('data_even', df_feat_alt["data_even"].cast(TimestampType()))\
#    .withColumn('label', df_feat_alt["label"].cast(DoubleType()))

In [168]:
#df_tidy.limit(10).toPandas()

In [169]:
#df_tidy.printSchema()

In [170]:
# salvando tabelao com label
#df_tidy.toPandas().to_csv(tabelao_final)

## Modelagem
O modelo usado será o de Random Forest, implementando pipelines da library ML do Spark.
A partir da amostra de dados será selecionado um subset com 75% dos dados para treinar o modelo. 

### Treino do Modelo
Prepara a amostra de acordo com o formato de entrada de label e vetor de features esperados pelo modelo.

In [324]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [325]:
# Transformar em double as features (features selecionadas)
# por algum motivo nao esta rodando no loop e nao á erro!!??
#for even in l_evenpoc:
#    df_tidy_dbl = df_tidy\
#        .withColumn(even, df_tidy[even].cast(DoubleType()))

df_tidy_dbl.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 5603: double (nullable = true)
 |-- 5605: double (nullable = true)
 |-- 5610: double (nullable = true)
 |-- 5612: double (nullable = true)
 |-- 5614: double (nullable = true)
 |-- 5616: double (nullable = true)
 |-- 5620: double (nullable = true)
 |-- 5622: double (nullable = true)
 |-- 5640: double (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- data_cad: timestamp (nullable = true)
 |-- atm: string (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)
 |-- max_evento: double (nullable = true)
 |-- label: double (nullable = false)



In [326]:
# rodando para as features com mais relevância já analisadas
#assemblerTrain = VectorAssembler(
#    inputCols=["655", "689", "1615", "1620", "1653", "2032", "2034", "2422", "2699", "2846",
#                "3009", "3010", "3011", "3321", "3367"],
#   outputCol="features")
assemblerTrain = VectorAssembler(
    inputCols=l_evenpoc,
   outputCol="features")

#               ,"1617", "1689","2433", "1746", "1568","883", "2163"],
 

In [327]:
df_tmp = assemblerTrain.transform(df_tidy_dbl)

In [328]:
df_tidy_dt = df_tmp.select("label", "features")
df_tidy_dt.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)



In [329]:
# Split the data into training and test sets (30% held out for testing)
(trainingData_rf, testData_rf) = df_tidy_dt.randomSplit([0.7, 0.3])

In [330]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
#labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df_tidy_dt)
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(trainingData_rf)

In [331]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 2 distinct values are treated as continuous.
#featureIndexer =\
#    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(df_tidy_dt)
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(trainingData_rf)

In [168]:
trainingData_rf.groupBy("label").count().orderBy("label").show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|  709|
|  1.0|    8|
+-----+-----+



In [169]:
testData_rf.groupBy("label").count().orderBy("label").show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|  256|
|  1.0|    4|
+-----+-----+



Separa a amostra de forma aleatória em dados de treino (70% da amostra) e de teste (30% da amostra, para posterior avaliação do modelo).

In [332]:
# Train a Random Forest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
#dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

In [333]:
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

In [334]:
model = pipeline.fit(trainingData_rf)

In [189]:
#pipeline = Pipeline(stages=[rf])
# Search through decision tree's maxDepth parameter for best model
#from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#from pyspark.ml.evaluation import BinaryClassificationEvaluator
#paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [2,3,4,5,6,7]).build()

#numFolds = 5

In [190]:
#crossval = CrossValidator(
#    estimator=pipeline,
#    estimatorParamMaps=paramGrid,
#    evaluator=evaluator,
#    numFolds=numFolds)

In [191]:
# Fetch best model
#tree_model = model.bestModel.stages[2]
#print tree_model

In [192]:
# Train model.  This also runs the indexers.
#model = pipeline.fit(trainingData_rf)

## Persistência do Modelo
Salva o modelo para posterior uso. O código comentado abaixo deve ser descomentado nas manutenções para testar se modelo foi corretamente salvo.

In [193]:
# SAVE MODEL FOR FUTURE USE (TEST IF SAVED OK)
#model.save(sc, savedModel) # Erro. Funciona com mllib. Pesquisar como salvar pipeline

### Avaliação do Modelo
Avalia o modelo usando os dados separados para teste, obtendo o erro de teste (1 - acurácia), a área sob a curva ROC (AUC) e a taxa Precision/Recall.

In [335]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions = model.transform(testData_rf)

In [336]:
evaluator = BinaryClassificationEvaluator()

In [337]:
evaluator.explainParams()

'labelCol: label column name (default: label)\nmetricName: metric name in evaluation (areaUnderROC|areaUnderPR) (default: areaUnderROC)\nrawPredictionCol: raw prediction (a.k.a. confidence) column name (default: rawPrediction)'

In [338]:
auroc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})
print("AUC = %g " % (auroc))

AUC = 1 


In [339]:
precrec = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})
print("Precision/Recall = %g " % (precrec))

Precision/Recall = 1 


In [340]:
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0 


In [341]:
predictions\
    .select("prediction", "indexedLabel", "features", "probability")\
    .groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       1.0|    1|
|       0.0|  215|
+----------+-----+



In [203]:
predictions.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- indexedLabel: double (nullable = true)
 |-- indexedFeatures: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [342]:
predictions.take(10)

[Row(label=0.0, features=DenseVector([1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0]), indexedLabel=0.0, indexedFeatures=DenseVector([1000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0]), rawPrediction=DenseVector([20.0, 0.0]), probability=DenseVector([1.0, 0.0]), prediction=0.0),
 Row(label=0.0, features=DenseVector([1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0]), indexedLabel=0.0, indexedFeatures=DenseVector([1000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0]), rawPrediction=DenseVector([20.0, 0.0]), probability=DenseVector([1.0, 0.0]), prediction=0.0),
 Row(label=0.0, features=DenseVector([1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0]), indexedLabel=0.0, indexedFeatures=DenseVector([1000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0]), rawPrediction=DenseVector([20.0, 0.0]), probability=DenseVector([1.0, 0.0]), prediction=0.0),
 Row(label=0.0, features=DenseVector([1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000

Abaixo são mostradas as primeiras linhas da tabela de previsão gerada pelo modelo

In [343]:
pd_pred = predictions.select("prediction", "indexedLabel", "features", "probability").toPandas()

In [186]:
# 6. dividir dados em treino e teste
# selecionando sample estratificada
#df_tidy_dbl.groupBy("label").count().orderBy("label").show()
# N: 0.9995   S: 0.000499
# Balanceamento com marcação anterior de report de POC
# N: 0.0110104 S: 0.98898
# Balanceamento com marcação por evento fora do percentil de POC
# N:  S: 

In [187]:
#df_tidy_dbl.groupBy(["nr_idef_secu", "label"]).count().orderBy("nr_idef_secu").show()

In [346]:
# salving the resulting dataframe
#type(poc_predicted)
pd_pred.to_csv(previsao)
#df.write.csv('mycsv.csv')

In [208]:
# estratégia
# 1. obter a média diária da agência + atm (trocar para mediana) para cada evento
# 2. do log, somar o número de ocorrências por agencia + atm + evento agrupado por dia
# 3. montar tabelao inicial e criar coluna count/media para ver índice de desvio da média (FEATURE) por agencia/atm
# 4. criar coluna LABEL concatenando com arquivo POC por agencia + data cad (atm nao confiavel no arquivo)
# obs: desvio > 1 indica percentual acima da média e vice-versa
# 5. fazer o pivot como acima

# trocar media por mediana (FALTA)
# usar mediana para gerar o tabelao (ou percentil 75% ou soma contante) (FALTA)
# dividir pelo count para ter a feature percentil/count (FALTA)
# transposicao (filtrar tabelao por agencia + atm e join dois a dois (em loop?))
# filtrar os 3 dias após a data de poc para tirar efeitos de setup   
# 

In [191]:
# matriz de confusão
#poc_predicted

## confusion matrix

In [205]:
def confusion_matrix(prevCol, labelCol):
    if prevCol == 1.0 and labelCol == 1.0:
        return 'TP'
    else:
        if prevCol == 1.0 and labelCol == 0.0:
            return 'FP'
        else:
            if prevCol == 0.0 and labelCol == 0.0:
                return 'TN'
            else:
                if prevCol == 0.0 and labelCol == 1.0:
                    return 'FN'
            
createNewColFromTwo = udf(confusion_matrix, DoubleType())
df_conf_matr = predictions.withColumn('conf_matr', createNewColFromTwo(predictions['prediction'],
            predictions['indexedLabel']))

In [207]:
df_conf_matr.show()

Py4JJavaError: An error occurred while calling o1396.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 606.0 failed 4 times, most recent failure: Lost task 9.3 in stage 606.0 (TID 42193, srvbdrprlbr06.santanderbr.corp): java.io.IOException: Unable to acquire 33554432 bytes of memory
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.acquireNewPage(UnsafeExternalSorter.java:351)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.<init>(UnsafeExternalSorter.java:138)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.create(UnsafeExternalSorter.java:106)
	at org.apache.spark.sql.execution.UnsafeExternalRowSorter.<init>(UnsafeExternalRowSorter.java:68)
	at org.apache.spark.sql.execution.TungstenSort.org$apache$spark$sql$execution$TungstenSort$$preparePartition$1(sort.scala:146)
	at org.apache.spark.sql.execution.TungstenSort$$anonfun$doExecute$3.apply(sort.scala:169)
	at org.apache.spar

In [199]:
# matriz de confuao
df_conf_matr.groupBy("conf_matr").count().show()

Py4JJavaError: An error occurred while calling o1355.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 570.0 failed 4 times, most recent failure: Lost task 10.3 in stage 570.0 (TID 40716, srvbdrprlbr05.santanderbr.corp): java.io.IOException: Unable to acquire 33554432 bytes of memory
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.acquireNewPage(UnsafeExternalSorter.java:351)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.<init>(UnsafeExternalSorter.java:138)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.create(UnsafeExternalSorter.java:106)
	at org.apache.spark.sql.execution.UnsafeExternalRowSorter.<init>(UnsafeExternalRowSorter.java:68)
	at org.apache.spark.sql.execution.TungstenSort.org$apache$spark$sql$execution$TungstenSort$$preparePartition$1(sort.scala:146)
	at org.apache.spark.sql.execution.TungstenSort$$anonfun$doExecute$3.apply(sort.scala:169)
	at org.apache.sp

In [194]:
# Sensibility - habilidade do modelo classificar corretamente um POC
# Probability of being test positive when POC present.
# TP / (TP + FN)
sensitivity = float(df_rates.collect()[0][1]) / (df_rates.collect()[0][1] + df_rates.collect()[2][1])
# Specificity - habilidade do modelo classificar corretamente a ausência de POC 
#Probability of being test negative when POC absent.
# TN / (TN + FP)
specificity = float(df_rates.collect()[1][1]) / (df_rates.collect()[1][1] + df_rates.collect()[3][1])
# precisao
# TP / (TP + FP)
precisao = float(df_rates.collect()[0][1]) / (df_rates.collect()[0][1] + df_rates.collect()[3][1])

print ("\nSentitividade ou Recall: " + str("%.2f" % (sensitivity)))
print ("\nEspecificidade         : " + str("%.2f" % (specificity)))
print ("\nPrecisão               : " + str("%.2f" % (precisao)))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 553.0 failed 4 times, most recent failure: Lost task 10.3 in stage 553.0 (TID 40035, srvbdrprlbr13.santanderbr.corp): java.io.IOException: Unable to acquire 33554432 bytes of memory
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.acquireNewPage(UnsafeExternalSorter.java:351)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.<init>(UnsafeExternalSorter.java:138)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.create(UnsafeExternalSorter.java:106)
	at org.apache.spark.sql.execution.UnsafeExternalRowSorter.<init>(UnsafeExternalRowSorter.java:68)
	at org.apache.spark.sql.execution.TungstenSort.org$apache$spark$sql$execution$TungstenSort$$preparePartition$1(sort.scala:146)
	at org.apache.spark.sql.execution.TungstenSort$$anonfun$doExecute$3.

In [195]:
df_rates.toPandas()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 558.0 failed 4 times, most recent failure: Lost task 10.3 in stage 558.0 (TID 40072, srvbdrprlbr13.santanderbr.corp): java.io.IOException: Unable to acquire 33554432 bytes of memory
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.acquireNewPage(UnsafeExternalSorter.java:351)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.<init>(UnsafeExternalSorter.java:138)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.create(UnsafeExternalSorter.java:106)
	at org.apache.spark.sql.execution.UnsafeExternalRowSorter.<init>(UnsafeExternalRowSorter.java:68)
	at org.apache.spark.sql.execution.TungstenSort.org$apache$spark$sql$execution$TungstenSort$$preparePartition$1(sort.scala:146)
	at org.apache.spark.sql.execution.TungstenSort$$anonfun$doExecute$3.

In [ ]:
# TESTES
predictions.filter((col("prediction") == 0.0) & (col("label") == 1.0)).count()

In [ ]:
df_rates.collect()

In [ ]:
predictions.filter((col("prediction") == 1.0) & (col("label") == 1.0)).count()

In [144]:
predictions.filter((col("prediction") == 1.0) & (col("label") == 0.0)).count()

621

In [107]:
predictions.filter((col("prediction") == 0.0) & (col("label") == 0.0)).count()

19698

In [145]:
# checagem do marcador de POC contra POCs reportados
# somente POC retencao de cartão, D-90
# trazer número de POCs para a chave agencia
# encotradas 52 agências com POC
df_p = df_poc.groupBy('agencia').count().orderBy('agencia')
df_p.toPandas()
# agora comparar com o arquivo gerado com label

agencia  count
0      065      1
1     1053      1
2     1074      2
3      112      2
4      118      1
5     1206      1
6     1217      1
7     1227      1
8     1320      1
9      140      1
10     144      1
11    1561      1
12    1610      1
13    1690      1
14     206      1
15    2082      2
16    2093      1
17     212      1
18    2175      1
19     220      2
20    2219      1
21    2226      1
22    2286      1
23     245      1
24     270      1
25    3063      1
26    3146      1
27    3165      1
28    3262      1
29    3282      2
30    3350      1
31     344      1
32    3544      1
33    3724      1
34    3733      1
35    4212      1
36    4237      1
37    4256      1
38    4263      1
39    4268      2
40    4355      1
41    4406      1
42    4550      2
43     458      1
44    4688      1
45     549      1
46     564      1
47     568      1
48     651      1
49     656      1
50     670      1
51     787      1
52     803      1

In [57]:
# foram encontradas agencias de POC nos eventos abaixo filtrados
df_l = df_feat.where(col('label') == 1.0).groupBy('nr_idef_secu').count().orderBy('nr_idef_secu')
#df_p = df_poc.groupBy('agencia').count().orderBy('agencia')
df_l.toPandas()

nr_idef_secu  count
0           105      1
1          1053      1
2          1074      1
3           112      2
4          1206      6
5          1217     10
6          1227      1
7          1320      1
8           140      4
9          1523      1
10         1530      1
11          154      1
12          156      1
13         1561      1
14         1640      2
15           17      6
16          177      3
17         2082      5
18         2092      4
19         2174      1
20         2175      1
21          220      1
22         2226      3
23         2286      4
24          238      4
25           24      7
26           26      3
27          262      2
28         3063      2
29         3146      2
..          ...    ...
38          344      1
39         3454      8
40         3544     12
41         3618      3
42         3650      1
43         3733      4
44          389      3
45         3912      1
46         3982     11
47         4212      1
48         4237      1
49         4256      1
50         4263      2
51          436      5
52         4406      3
53         4428      5
54         4433      4
55         4550      3
56         4688      1
57         4728      1
58         4776      1
59          568      1
60           65      1
61          652      1
62          670      3
63          678      6
64          690      2
65          734      3
66           86      1
67          990      1

[68 rows x 2 columns]

In [58]:
# checar se selecionou realmente somente agencias com POC do log geral
df_evenpoc.groupBy('nr_idef_secu').count().orderBy('nr_idef_secu').toPandas()

nr_idef_secu  count
0           3650     11
1           4776      9
2           3454     25
3           1207      1
4            154     10
5            112     41
6           4355      7
7            652     17
8           3733     55
9            670     38
10           389     28
11          1530     16
12          3544    110
13          3063     24
14          1227     28
15          2014      3
16           787     23
17          1074     18
18           644     12
19           188     12
20           568     13
21           458      5
22          3310     17
23           656     33
24          4256     15
25          3350     65
26          3618     19
27          3724     10
28          1610      3
29          4237     10
..           ...    ...
78           803      2
79           105     27
80          3265     47
81          4433     52
82          2219      2
83          1690      8
84          4728     15
85          2047      7
86            65     50
87           734     18
88          4428     34
89          3055     45
90          1525     11
91          2286     46
92           690     34
93          2093     14
94            21     20
95           651      4
96          2226     34
97           151      8
98           262     25
99          3416     23
100         4688      5
101         3371     40
102         3396     10
103         1717      1
104         4268      2
105         3974      5
106          990     14
107         3412     40

[108 rows x 2 columns]

In [67]:
# checar quantas agencias reportadas como POC cairam no log com nova marcação (deveriam aparecer todos )
df_j = df_l \
    .join(df_p, (df_l['nr_idef_secu'] == df_p['agencia']), 'inner')
    
    

In [68]:
df_j.limit(10).toPandas()

nr_idef_secu  count agencia  count
0          112      2     112      2
1         3733      4    3733      1
2          670      3     670      1
3         3544     12    3544      1
4         1227      1    1227      1
5         3063      2    3063      1
6         1074      1    1074      2
7          568      1     568      1
8         4256      1    4256      1
9         3350      2    3350      1